In [103]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [104]:
df=pd.read_csv('train.csv')
df_2=pd.read_csv('test.csv')
df_all=pd.concat([df,df_2],sort=False)
df_all = df_all.iloc[:, :-1]

In [105]:
#name前导处理，只保留‘ ’头衔部分，注意这里有空格，下面也要有
df_all['Name'] = df_all['Name'].str.split(',').str[1].str.split('.').str[0]


#处理age的缺失值
x = df_all["Age"].median()
df_all["Age"].fillna(x, inplace=True)


#处理Sex的表述
df_all['Sex'] = df['Sex'].replace({'male': 0, 'female': 1})


#处理Nmae的表述，注意这里有空格
df_all['Name']=df_all['Name'].replace({' Mr': 0, ' Mrs': 1, ' Miss': 2,' Master': 3, ' Don': 4, ' Rev': 5, ' Dr': 6, ' Mme': 7, ' Ms': 8, ' Major': 9, ' Lady': 10, ' Sir': 11, ' Mlle': 12, ' Col': 13, ' Capt': 14, ' the Countess': 15, ' Jonkheer': 16, ' Dona': 17})


#处理Embarked的表述
df_all['Embarked']=df_all['Embarked'].replace({'S': 0, 'C': 1, 'Q': 2})


#删除Ticket和Cabin
df_all.drop(['Ticket', 'Cabin'], axis=1, inplace=True)

print(df_all)

     PassengerId  Survived  Pclass  Name  Sex   Age  SibSp  Parch      Fare  \
0              1       0.0       3     0    0  22.0      1      0    7.2500   
1              2       1.0       1     1    1  38.0      1      0   71.2833   
2              3       1.0       3     2    1  26.0      0      0    7.9250   
3              4       1.0       1     1    1  35.0      1      0   53.1000   
4              5       0.0       3     0    0  35.0      0      0    8.0500   
..           ...       ...     ...   ...  ...   ...    ...    ...       ...   
413         1305       NaN       3     0    0  28.0      0      0    8.0500   
414         1306       NaN       1    17    0  39.0      0      0  108.9000   
415         1307       NaN       3     0    1  38.5      0      0    7.2500   
416         1308       NaN       3     0    1  28.0      0      0    8.0500   
417         1309       NaN       3     3    1  28.0      1      1   22.3583   

     Embarked  
0         0.0  
1         1.0  
2  

C:\Users\admin\AppData\Local\Temp\ipykernel_13636\3248198929.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_all["Age"].fillna(x, inplace=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_13636\3248198929.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_all['Sex'] = df['Sex'].replace({

In [106]:
#数据正则化
scaler = MinMaxScaler()
df_all = scaler.fit_transform(df_all)

#划分数据，还原训练集和测试集
array_trian=df_all[:891]
array_test=df_all[891:]
mask = ~np.isnan(array_trian).any(axis=1)
# 删除包含 NaN 值的行
array_trian = array_trian[mask]
#切出标签
y_train= array_trian[:,1]
y_train=y_train.astype(int)
X_train = np.delete(array_trian, 1, axis=1)
x_test = np.delete(array_test, 1, axis=1)
col_mean = np.nanmean(x_test, axis=0)

# 查找 NaN 值的位置
inds = np.where(np.isnan(x_test))

# 将 NaN 值替换为对应列的平均值
x_test[inds] = np.take(col_mean, inds[1])
np.savetxt('X_train.csv', X_train, delimiter=',')
np.savetxt('y_train.csv', y_train, delimiter=',')
np.savetxt('x_test.csv', x_test, delimiter=',')

# 不同算法比较
我滚回来调优了，现在决定每个算法都试一试，看看哪个效果比较好，首先是逻辑回归，kaggle得分0.555

In [107]:
# model = LogisticRegression()
# model.fit(X_train, y_train)
# y_pred = model.predict(x_test)
# y_pred = y_pred.astype(int)
# length = len(y_pred)
# print(type(y_pred))
# print(y_pred)
# print(length)

接下来是K近邻投票,在3邻居的情况下收获了0.590的评分！

在8邻居的情况下直接收获0.607的最高得分！

但是优化的极限差不多也到这里就结束了，再往后变成20邻居也没有质的提升了

In [108]:
# model = KNeighborsClassifier(n_neighbors=20)
# model.fit(X_train, y_train)
# y_pred = model.predict(x_test)

让我们运行一下支持向量机，很遗憾，只有0.535的准确率，甚至不如逻辑回归

 ~~可惜不是你，陪我到最后~~

In [109]:
# model = SVC(kernel='linear')
# model.fit(X_train, y_train)
# y_pred = model.predict(x_test)

下面登场的是随机森林算法！！！

先让我们看看100棵树，得分0.739

在1000棵树的时候，出现了很有意思的情况，过拟合！得分0.736

那么50棵树会发生什么？欠拟合，得分只剩下0.70了

In [110]:
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)
rf_model = RandomForestClassifier(n_estimators=100)
rf_model.fit(X_train, y_train)
dt_pred = dt_model.predict(x_test)
rf_pred = rf_model.predict(x_test)
y_pred=rf_pred

## 总结
写一个好算法固然很重要，但是洗好数据更重要，数据的样本量和处理方式才是更值得讨论的话题


In [111]:
pd.DataFrame({
     'PassengerId': df_2.PassengerId,
     'Survived' : y_pred
 }).to_csv('submission.csv', index = False)